In [248]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as plt_io
import plotly.express as px
import plotly.figure_factory as ff
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler


In [249]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from config import DB_USER, DB_PASSWORD, DB_SERVER_NAME, DB_DATABASE_NAME
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = DB_USER,
    password = DB_PASSWORD,
    host = DB_SERVER_NAME, 
    port = 5432,
    database = DB_DATABASE_NAME, 
)

In [250]:
engine = create_engine(connection_url)

In [251]:
BTC_df = pd.read_sql("btc", engine)

In [252]:
BTC_df=BTC_df.set_index('Date')

In [253]:
BTC_df

,Open,High,Low,Close,Adj Close,Volume,ds,y
Date,,,,,,,,
2014-10-15,400.954987,402.226990,388.765991,394.773010,394.773010,2.526710e+07,2014-10-15,394.773010
2017-03-01,1180.040039,1222.500000,1179.689941,1222.500000,1222.500000,2.290570e+08,2017-03-01,1222.500000
2014-10-16,394.518005,398.807007,373.070007,382.556000,382.556000,2.699000e+07,2014-10-16,382.556000
2014-10-17,382.756012,385.477997,375.389008,383.757996,383.757996,1.360070e+07,2014-10-17,383.757996
2014-10-28,353.214996,359.984009,352.678986,357.618012,357.618012,7.845880e+06,2014-10-28,357.618012
...,...,...,...,...,...,...,...,...
2022-02-17,43937.070310,44132.972660,40249.371090,40538.011720,40538.011720,2.624666e+10,2022-02-17,40538.011720
2022-02-18,40552.132810,40929.152340,39637.617190,40030.976560,40030.976560,2.331001e+10,2022-02-18,40030.976560
2022-02-19,40026.023440,40418.878910,39713.058590,40122.156250,40122.156250,1.373656e+10,2022-02-19,40122.156250


In [254]:
BTC_df = BTC_df[['Close']]

In [255]:
latest_df = BTC_df.tail(365)

In [256]:
latest_df = latest_df.dropna()

In [257]:
latest_df

,Close
Date,
2021-02-22,54207.32031
2021-02-23,48824.42578
2021-02-24,49705.33203
2021-02-25,47093.85156
2021-02-26,46339.76172
...,...
2022-02-17,40538.01172
2022-02-18,40030.97656
2022-02-19,40122.15625


In [258]:
train_rec = int(len(latest_df)*0.70)

In [259]:
train_df = latest_df[0:train_rec]
print(train_df.shape)

(255, 1)


In [260]:
test_df = latest_df[train_rec:]
print(test_df.shape)


(110, 1)


In [261]:
sc = MinMaxScaler(feature_range = (0, 1))
train_scaled = sc.fit_transform(train_df)
test_scaled =sc.transform(test_df)

def dataset(data , n_features):
    X, Y = [],[]
    for i in range(len(data)-n_features-1):
        a=data[i:(i+n_features),0]

        X.append(a)
        Y.append(data[i+n_features,0])
    return np.array(X), np.array(Y)

n_features = 3
X_train, y_train = dataset(train_scaled, n_features)
X_test, y_test = dataset(test_scaled, n_features)



C:\Users\asfia\anaconda3\envs\crypto-analysis\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.

C:\Users\asfia\anaconda3\envs\crypto-analysis\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.

C:\Users\asfia\anaconda3\envs\crypto-analysis\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.



In [262]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
print(X_train.shape , y_train.shape, X_test.shape, y_train.shape) 

(251, 3, 1) (251,) (106, 3, 1) (251,)


In [263]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM


In [282]:
inputs = keras.layers.Input(shape=(X_test.shape[1], X_test.shape[2]) )
x = keras.layers.LSTM(30,return_sequences=True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(30,return_sequences= True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(30)(x)
outputs = keras.layers.Dense(1, activation = 'linear')(x)

model  = keras.Model(inputs=inputs,outputs=outputs)






In [283]:

model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 3, 1)]            0         
                                                                 
 lstm_42 (LSTM)              (None, 3, 30)             3840      
                                                                 
 dropout_21 (Dropout)        (None, 3, 30)             0         
                                                                 
 lstm_43 (LSTM)              (None, 30)                7320      
                                                                 
 dense_13 (Dense)            (None, 1)                 31        
                                                                 
Total params: 11,191
Trainable params: 11,191
Non-trainable params: 0
_________________________________________________________________


In [284]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [267]:
X_test.shape

(106, 3, 1)

In [286]:
history = model.fit(X_test,y_test,epochs=25,batch_size = 60,validation_split  = 0)
pred=model.predict(X_test)
pred=sc.inverse_transform(pred)

pred.shape


Epoch 1/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0145 - accuracy: 0.0000e+00
Epoch 2/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0165 - accuracy: 0.0000e+00
Epoch 3/25
2/2 [==============================] - 0s 6ms/step - loss: 0.0134 - accuracy: 0.0000e+00
Epoch 4/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0150 - accuracy: 0.0000e+00
Epoch 5/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0132 - accuracy: 0.0000e+00
Epoch 6/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0126 - accuracy: 0.0000e+00
Epoch 7/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0149 - accuracy: 0.0000e+00
Epoch 8/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0143 - accuracy: 0.0000e+00
Epoch 9/25
2/2 [==============================] - 0s 5ms/step - loss: 0.0120 - accuracy: 0.0000e+00
Epoch 10/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0106 - accuracy: 0.0000e+00

(106, 1)

In [287]:
test_actual = y_test.reshape(y_test.shape[0],1)
test_actual = sc.inverse_transform(test_actual)
test_actual.shape


(106, 1)

In [288]:
test_predicted = []
for i in pred:
    test_predicted.append(i[0])
df_predicted = pd.DataFrame()
df_predicted['Prediction'] = test_predicted
df_predicted ['Actual'] = test_actual
df_predicted.head()

,Prediction,Actual
0,58007.601562,63326.98828
1,58249.046875,67566.82813
2,59518.488281,66971.82813
3,61346.117188,64995.23047
4,62597.609375,64949.96094


In [289]:
def interactive_plot(df,title):
    stock_chart = px.line(title=title)
    for i in df.columns[0:]:
        stock_chart.add_scatter(x= df.index,y=df[i], name = i)
    stock_chart.show()
interactive_plot(df_predicted,"Stock Price Predicted")

In [290]:
df_predicted['index'] = df_predicted.index


In [292]:
df_predicted


,Prediction,Actual,index
0,58007.601562,63326.98828,0
1,58249.046875,67566.82813,1
2,59518.488281,66971.82813,2
3,61346.117188,64995.23047,3
4,62597.609375,64949.96094,4
...,...,...,...
101,44109.542969,43961.85938,101
102,44619.058594,40538.01172,102
103,44713.320312,40030.97656,103
104,43672.363281,40122.15625,104


In [305]:


df_predicted.to_csv('predicted2.csv', encoding='utf-8',index='true')